In [11]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

True

In [12]:
from crewai_tools import tool

import anthropic
from openai import OpenAI
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import requests
import time
import os
from requests.exceptions import RequestException
import logging
import random
from pydub import AudioSegment
import subprocess
from typing import Dict
import os
import uuid
import os
from google.cloud import texttospeech
import uuid
from pydub import AudioSegment
import subprocess
from typing import List, Dict
import textwrap
import requests
import random

tts_client = texttospeech.TextToSpeechClient()
client = anthropic.Anthropic(api_key="")

# Google Cloud 인증 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/chloepark/Downloads/auto-generation-428209-ad2beb3ee8ba.json"

class Tools:

    @staticmethod
    def mix_audio(voice_files: List[str], bg_music_file: str, sfx_files: List[str], output_file: str):
        try:
            # 임시 파일들
            intro_file = "temp_intro.mp3"
            outro_file = "temp_outro.mp3"
            
            # 인트로 배경음악 생성 (10초, 페이드아웃)
            subprocess.run([
                'ffmpeg', '-y',
                '-i', bg_music_file,
                '-t', '10',
                '-af', 'afade=t=out:st=5:d=5',
                '-c:a', 'libmp3lame',
                intro_file
            ], check=True, capture_output=True)

            # 아웃트로 배경음악 생성 (10초, 페이드인 후 페이드아웃)
            subprocess.run([
                'ffmpeg', '-y',
                '-i', bg_music_file,
                '-t', '10',
                '-af', 'afade=t=in:st=0:d=5,afade=t=out:st=5:d=5',
                '-c:a', 'libmp3lame',
                outro_file
            ], check=True, capture_output=True)

            # 모든 오디오 파일 목록 생성
            all_files = [intro_file] + voice_files + sfx_files + [outro_file]
            
            # 파일 목록을 ffmpeg 입력으로 변환
            input_args = []
            for file in all_files:
                input_args.extend(['-i', file])

            # 최종 오디오 믹싱
            filter_complex = f'[0:a][1:a]'
            for i in range(2, len(all_files)):
                filter_complex += f'[{i}:a]'
            filter_complex += f'concat=n={len(all_files)}:v=0:a=1[out]'

            final_command = [
                'ffmpeg', '-y'
            ] + input_args + [
                '-filter_complex', filter_complex,
                '-map', '[out]',
                '-c:a', 'libmp3lame',
                '-q:a', '2',
                output_file
            ]
            subprocess.run(final_command, check=True, capture_output=True)
            
            logging.info(f"Audio mixing completed successfully: {output_file}")
            return output_file
        except subprocess.CalledProcessError as e:
            logging.error(f"Error during audio mixing: {e.stderr}")
            raise Exception(f"Failed to mix audio: {e.stderr}")
        finally:
            # 임시 파일 삭제
            for temp_file in [intro_file, outro_file] + voice_files + sfx_files:
                if os.path.exists(temp_file):
                    os.remove(temp_file)

    @staticmethod
    def generate_sfx(text: str, duration_seconds: int, prompt_influence: float = 0.5) -> str:
        """
        Generates sound effects that match the given text using the Elevenlabs API.

        Args:
        text (str): The text to generate sound effects for.
        duration_seconds (int): The duration of the sound effect in seconds.
        prompt_influence (float): The influence of the prompt on the generated sound effect. Default is 0.5.

        Returns:
        str: The file path of the generated sound effect.
        """
        api_key = ""
        if not api_key:
            logging.warning("ELEVENLABS_API_KEY environment variable is not set")
            return None
        
        url = "https://api.elevenlabs.io/v1/sound-generation"
        headers = {
            "Content-Type": "application/json",
            "xi-api-key": api_key
        }

        # 텍스트를 450자 이하의 청크로 나누기
        chunks = textwrap.wrap(text, 450)
        sfx_files = []

        try:
            for chunk in chunks:
                payload = {
                    "text": chunk,
                    "duration_seconds": duration_seconds // len(chunks),  # 각 청크에 대한 지속 시간을 균등하게 분배
                    "prompt_influence": prompt_influence
                }

                response = requests.post(url, json=payload, headers=headers)
                if response.status_code != 200:
                        logging.error(f"Error generating sound effects: {response.status_code}, {response.text}")
                        return None

                sfx_file = f"{uuid.uuid4()}.mp3"
                with open(sfx_file, "wb") as f:
                    f.write(response.content)
                sfx_files.append(sfx_file)
            
            # 여러 개의 사운드 이펙트 파일을 하나로 합치기
            if len(sfx_files) > 1:
                combined_sfx = f"{uuid.uuid4()}.mp3"
                with AudioSegment.from_mp3(sfx_files[0]) as combined:
                    for sfx_file in sfx_files[1:]:
                        combined += AudioSegment.from_mp3(sfx_file)
                    combined.export(combined_sfx, format="mp3")
                
                # 임시 파일 삭제
                for sfx_file in sfx_files:
                    os.remove(sfx_file)
                
                return combined_sfx
            elif sfx_files:
                return sfx_files[0]
            else:
                return None
        except Exception as e:
            logging.error(f"Error in generate_sfx: {str(e)}")
            return None
    
    @staticmethod
    def generate_audio_chunk(text: str, voice_name: str = "en-US-Neural2-F") -> str:
        client = texttospeech.TextToSpeechClient()

        synthesis_input = texttospeech.SynthesisInput(text=text)

        voice = texttospeech.VoiceSelectionParams(
            language_code="en-US",
            name=voice_name
        )

        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3
        )

        try:
            response = tts_client.synthesize_speech(
                input=synthesis_input, voice=voice, audio_config=audio_config
            )

            voice_file = f"{uuid.uuid4()}.mp3"
            with open(voice_file, "wb") as out:
                out.write(response.audio_content)
            
            return voice_file
        except Exception as e:
            logging.error(f"Error generating audio chunk: {str(e)}")
            raise

    @staticmethod
    @tool("Generate Story Audio")
    def generate_audio(story_text: str) -> str:
        """
#         Converts a given story text into an audio file using the Google Cloud TTS API.
#         The input should be a string of the story text.
        
#         This function breaks down the input text into chunks, generates audio for each chunk,
#         creates sound effects, and then mixes all the audio components together with 
#         background music to create a final audio file.

#         Args:
#         story_text (str): The full text of the story to be converted to audio.

#         Returns:
#         str: The file path of the generated audio file.
#         """
        chunk_size = 1000
        chunks = [story_text[i:i+chunk_size] for i in range(0, len(story_text), chunk_size)]

        voice_params = texttospeech.VoiceSelectionParams(
            language_code="en-US",  # Ensure this matches the language code
            name="en-US-Neural2-D",  # Ensure this matches the name from list_voices output
            ssml_gender=texttospeech.SsmlVoiceGender.MALE
        )

        audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
        )

        voice_files = []
        sfx_files = []

        for chunk in chunks:
            synthesis_input = texttospeech.SynthesisInput(text=chunk)
            try:
                response = tts_client.synthesize_speech(
                input=synthesis_input, voice=voice_params, audio_config=audio_config
            )

                voice_file_path = f"voice_chunk_{chunks.index(chunk)}.mp3"
                with open(voice_file_path, "wb") as out:
                    out.write(response.audio_content)
                voice_files.append(voice_file_path)
                
                sfx_file = Tools.generate_sfx(chunk, 5, 0.5)
                if sfx_file:
                        sfx_files.append(sfx_file)
            except Exception as e:
                logging.error(f"Error processing chunk: {str(e)}")
                continue
            
        bg_music_files = [
            "/Users/chloepark/Downloads/romance1.mp3",
            "/Users/chloepark/Downloads/romance2.mp3",
            "/Users/chloepark/Downloads/romance3.mp3",
            "/Users/chloepark/Downloads/romance4.mp3",
            "/Users/chloepark/Downloads/romance5.mp3",
            "/Users/chloepark/Downloads/romance6.mp3",
            "/Users/chloepark/Downloads/romance7.mp3",
            "/Users/chloepark/Downloads/romance8.mp3",
            "/Users/chloepark/Downloads/romance9.mp3",
            "/Users/chloepark/Downloads/romance10.mp3",
            "/Users/chloepark/Downloads/romance11.mp3",
            "/Users/chloepark/Downloads/romance12.mp3",
            "/Users/chloepark/Downloads/romance13.mp3",
            "/Users/chloepark/Downloads/romance14.mp3",
            "/Users/chloepark/Downloads/romance15.mp3",
        ]
        
        bg_music_file = random.choice(bg_music_files)
        output_file = "final_output_with_intro_outro.mp3"

        # sfx_file = Tools.generate_sfx(story_text, 10, 0.5)

        return Tools.mix_audio(voice_files, bg_music_file, sfx_files, output_file)
    
    @tool("Generate Story Image")
    def generate_image(story_text: str) -> Dict[str, str]:
            """
            Creates an image based on a given description using OpenAI's DALL-E.
        The input should be a string describing the image.
        Returns a dictionary containing the base64 encoded image data and the file path.
            """

            # openai.api_key = "OPENAI_API_KEY"
            # client = OpenAI()
            client = OpenAI(api_key="")
            response = client.images.generate(
                prompt=story_text,
                n=1,
                size="1024x1024",
                response_format="b64_json" #추가
            )

            # image_url = response.data[0].url
            image_data = response.data[0].b64_json #추가
            
            image_file_path = "output.png"
            import base64 #추가
            with open(image_file_path, "wb") as f:
                f.write(base64.b64decode(image_data)) #추가
            return image_file_path   
    

In [13]:

from crewai import Agent

class Agents:
    def __init__(self):
    #     try:
        self.tools = Tools()
        # except Exception as e:
        #     print(f"Error initializing Tools: {e}")
        #     self.tools = None

    def script_writer(self):
        return Agent(
            role="Script Writer",
            goal="Writing a captivating 10000-word romance story for sophisticated women in their 30s and 40s.",
            backstory="You are an award-winning novelist specializing in passionate romance stories. Your writing is known for its emotional depth, sensual tension, and complex characters that resonate with mature readers.",
            verbose=True,
            allow_delegation=False,
            tools=[]
        )

    def illustrator(self):
        return Agent(
            role="Illustrator",
            goal="Creating an evocative and sophisticated thumbnail for the romance story.",
            backstory="You are a renowned illustrator celebrated for your ability to capture complex emotions and intimate moments in your art. Your work is known for its sensual yet tasteful style, appealing to mature, discerning readers.",
            verbose=True,
            allow_delegation=False,
            tools=[
                self.tools.generate_image,
            ]
        )

    def voice_actor(self):
        return Agent(
            role="Voice Actor",
            goal="Creating an immersive audio experience for the romance story using Google TTS and background music.",
            backstory="You are an audio production specialist skilled in using Google TTS and audio editing to create captivating audiobooks. Your expertise lies in selecting the perfect voice, adjusting pacing, and integrating background music to enhance the listener's experience.",
            verbose=True,
            allow_delegation=False,
            tools=[
                self.tools.generate_audio,
            ]
        )

    def metadata_writer(self):
        return Agent(
            role="Metadata Writer",
            goal="Creating comprehensive and engaging metadata for the romance story audiobook.",
            backstory="You are a metadata specialist with a keen understanding of audiobook marketing. Your skill lies in crafting compelling titles, concise yet intriguing descriptions, and accurately documenting technical details to enhance discoverability and appeal.",
            verbose=True,
            allow_delegation=False,
            tools=[]
        )

In [14]:
from crewai import Task

class Tasks:
    def __init__(self):
        self.script_task = None
        self.thumbnail_task = None
        self.audio_task = None
        self.metadata_task = None

    def create_script_task(self, agent):
        self.script_task = Task(
            description=
            """
            Write a 5000-word romance novel about {subject} for American housewives in their 30s and 40s. The novel should be engaging, romantic, and include dialogues. All sentences must end with a period. 
            The story should be romantic and dramatic, with vivid descriptions and emotional depth. It should not be a description or scenario of a story. It should be written in English and be original, not copied from any other source.
            """,
            expected_output=
            """ 
            Your answer MUST be a 5000-word romance story for housewives in America. The story should be engaging, romantic, and include dialogues. 
            It should be in English. All sentences must end with a period. It should be a short story itself. The story should be original and not copied from any other source.
            It should not be a description or scenario of a story. It should not include chapter titles or author's notes.
            """,
            agent=agent,
            output_file="script.md",
        )
        return self.script_task

    def create_thumbnail_task(self, agent):
        self.thumbnail_task = Task(
            description="Create a thumbnail for the romance novel script.",
            expected_output="Generate a PNG image file for the romance novel script thumbnail. The image should be engaging, romantic, and capture the essence of the story. The characters should be attractive. Save the image directly as 'output.png'. Do not return any text or markdown. The task is complete when the image file is saved.",
            agent=agent,
            context=[self.script_task],
            output_file="thumbnail.png",
        )
        return self.thumbnail_task

    def create_audio_task(self, agent):
        self.audio_task = Task(
            description="Generate an immersive audio recording of the romance novel using Google TTS and add background music.",
            expected_output="""
            1. Use the Google TTS API to generate an audio recording of the romance novel script.
            2. Select an appropriate American male voice for the narration.
            3. Adjust the TTS settings to insert a 0.3-second pause between sentences for natural pacing.
            4. Generate the audio without reading the title.
            5. Add subtle, mood-appropriate background music to the beginning and end of the recording.
            6. Ensure smooth transitions between narration and music.
            7. Save the final audio file as 'final_output_with_intro_outro.mp3'.
            Do not return any text or markdown. The task is complete when the audio file is saved.
            """,
            agent=agent,
            context=[self.script_task],
            output_file="audio.mp3",
        )
        return self.audio_task

    def create_metadata_task(self, agent):
        self.metadata_task = Task(
            description="Generate a comprehensive metadata file for the romance story audiobook.",
            expected_output="""
            Create a markdown file containing the following metadata for the romance story audiobook:
            1. Title: An engaging title that captures the essence of the story.
            2. Description: A compelling 100-word summary of the story, highlighting key themes and emotional appeal.
            3. Author: The name of the AI model that generated the story.
            4. Narrator: The Google TTS voice ID used for narration.
            5. Duration: The total runtime of the audiobook.
            6. Genre: Specify the subgenre of romance (e.g., contemporary, historical, paranormal).
            7. Target Audience: Brief description of the intended audience.
            8. Content Rating: Indicate the maturity level of the content.
            9. Keywords: 5-7 relevant keywords for improved searchability.
            10. Production Notes: Brief mention of AI story generation and TTS narration.
            All information should be in English and formatted clearly in markdown.
            """,
            agent=agent,
            context=[
                self.script_task,
                self.audio_task,
            ],
            output_file="metadata.md",
        )
        return self.metadata_task

In [15]:
from crewai import Crew
import mimetypes

# Slack 클라이언트 설정
slack_token = os.environ["SLACK_BOT_TOKEN"]
slack_client = WebClient(token=slack_token)
slack_channel_id = "C07CEMU9N7J"  # 실제 채널명으로 교체해야 합니다

def post_to_slack(slack_channel_id, message, file_path=None):
    try:
        # 텍스트 메시지 게시
        if message:
            slack_client.chat_postMessage(channel=slack_channel_id, text=message)
        
        # 파일 업로드 (파일이 존재하는 경우에만)
        if file_path and os.path.exists(file_path):
            mime_type, encoding = mimetypes.guess_type(file_path)
            if mime_type is None:
                mime_type = 'application/octet-stream'
            
            with open(file_path, "rb") as file_content:
                response = slack_client.files_upload_v2(
                    channel=slack_channel_id,
                    file=file_content,
                    # content=file_content.read(),
                    filename=os.path.basename(file_path),
                    
                )
            print(f"Successfully uploaded file: {file_path}")
            return True
        elif file_path:
            print(f"File not found: {file_path}")
            return False
    except SlackApiError as e:
        print(f"Error posting to Slack: {e}")
        return False
    return True

agents = Agents()
tasks = Tasks()

script_writer = agents.script_writer()
illustrator = agents.illustrator()
voice_actor = agents.voice_actor()
metadata_writer = agents.metadata_writer()

# 엑셀 파일에서 키워드 읽기
excel_file_path = '/Users/chloepark/Downloads/romance_keywords.xlsx'
df = pd.read_excel(excel_file_path)

#열 이름 확인 및 키워드 추출
if 'Keywords' in df.columns:
    keywords = df['Keywords'].tolist()
elif 'Keyword' in df.columns:
    keywords = df['Keyword'].tolist()
else:
    print("Error: 'Keywords' or 'Keyword' column not found in the Excel file.")
    print("Available columns:", df.columns.tolist())
    raise ValueError("Required column not found in Excel file")

for keyword in keywords:
    script_task = tasks.create_script_task(script_writer)
    thumbnail_task = tasks.create_thumbnail_task(illustrator)
    audio_task = tasks.create_audio_task(voice_actor)
    metadata_task = tasks.create_metadata_task(metadata_writer)



    crew = Crew(
        agents=[
            script_writer,
            illustrator,
            voice_actor,
            metadata_writer,
        ],
        tasks=[
            script_task,
            thumbnail_task,
            audio_task,
            metadata_task,
        ],
        verbose=2,
    )


    try:
        result = crew.kickoff(
            inputs=dict(
                subject=keyword,
            )
        )
        
        print(f"Completed tasks for keyword: {keyword}")
        
        # 결과물 Slack에 게시 (성공한 항목만)
        post_to_slack(slack_channel_id, f"키워드: {keyword}")
        
        files_to_upload = [
            ("스크립트", "script.md"),
            ("썸네일", "output.png"),
            ("오디오", "final_output_with_intro_outro.mp3"),
            ("메타데이터", "metadata.md")
        ]
        
        for description, file_path in files_to_upload:
            if description == "스크립트":
                post_to_slack(slack_channel_id, "스크립트:")
                if post_to_slack(slack_channel_id, None, file_path):
                    print(f"Successfully uploaded 스크립트")
                else:
                    print(f"Failed to upload 스크립트")
                time.sleep(2)
            elif description == "썸네일":
                post_to_slack(slack_channel_id, "썸네일:")
                if post_to_slack(slack_channel_id, None, file_path):
                    print(f"Successfully uploaded 썸네일")
                else:
                    print(f"Failed to upload 썸네일")
                time.sleep(2)
            elif description == "오디오":
                post_to_slack(slack_channel_id, "오디오:")
                if post_to_slack(slack_channel_id, None, file_path):
                    print(f"Successfully uploaded 오디오")
                else:
                    print(f"Failed to upload 오디오")
                time.sleep(2)
            elif description == "메타데이터":
                post_to_slack(slack_channel_id, "메타데이터:")
                if post_to_slack(slack_channel_id, None, file_path):
                    print(f"Successfully uploaded 메타데이터")
                else:
                    print(f"Failed to upload 메타데이터")
                time.sleep(2)
        
        print(f"Posted results for keyword: {keyword} to Slack")

    except Exception as e:
        error_message = f"Unexpected error processing keyword '{keyword}': {str(e)}"
        print(error_message)
        post_to_slack(slack_channel_id, error_message)
    print("\n---\n")
    
    # 5초 대기
    time.sleep(5)

print("All tasks completed.")


 [DEBUG]: == Working Agent: Script Writer
 [INFO]: == Starting Task: 
            Write a 5000-word romance novel about A woman finds love with her physical therapist. for American housewives in their 30s and 40s. The novel should be engaging, romantic, and include dialogues. All sentences must end with a period. 
            The story should be romantic and dramatic, with vivid descriptions and emotional depth. It should not be a description or scenario of a story. It should be written in English and be original, not copied from any other source.
            


> Entering new CrewAgentExecutor chain...
I now have a clear vision of the story. I can see our protagonist, a woman in her mid-thirties, who has been through a traumatic event that requires physical therapy. I can see the charming, empathetic physical therapist who enters her life and helps her heal in more ways than one. The setting, the characters, the dialogue... everything is ready to unfold in my mind.

Final Answer:

Sam

2024-09-12 18:50:06,218 - 8274185216 - 3381157008.py-3381157008:114 - WARNING: ELEVENLABS_API_KEY environment variable is not set
2024-09-12 18:50:08,747 - 8274185216 - 3381157008.py-3381157008:114 - WARNING: ELEVENLABS_API_KEY environment variable is not set
2024-09-12 18:50:10,285 - 8274185216 - 3381157008.py-3381157008:114 - WARNING: ELEVENLABS_API_KEY environment variable is not set


 

final_output_with_intro_outro.mp3



KeyboardInterrupt: 

In [ ]:
from google.cloud import texttospeech

def list_voices():
    client = texttospeech.TextToSpeechClient()
    response = client.list_voices()

    voices = response.voices
    for voice in voices:
        languages = ", ".join(voice.language_codes)
        print(f"Name: {voice.name}")
        print(f"Languages: {languages}")
        print(f"SSML Gender: {texttospeech.SsmlVoiceGender(voice.ssml_gender).name}")
        print("=" * 40)

list_voices()

Name: af-ZA-Standard-A
Languages: af-ZA
SSML Gender: FEMALE
Name: am-ET-Standard-A
Languages: am-ET
SSML Gender: FEMALE
Name: am-ET-Standard-B
Languages: am-ET
SSML Gender: MALE
Name: am-ET-Wavenet-A
Languages: am-ET
SSML Gender: FEMALE
Name: am-ET-Wavenet-B
Languages: am-ET
SSML Gender: MALE
Name: ar-XA-Standard-A
Languages: ar-XA
SSML Gender: FEMALE
Name: ar-XA-Standard-B
Languages: ar-XA
SSML Gender: MALE
Name: ar-XA-Standard-C
Languages: ar-XA
SSML Gender: MALE
Name: ar-XA-Standard-D
Languages: ar-XA
SSML Gender: FEMALE
Name: ar-XA-Wavenet-A
Languages: ar-XA
SSML Gender: FEMALE
Name: ar-XA-Wavenet-B
Languages: ar-XA
SSML Gender: MALE
Name: ar-XA-Wavenet-C
Languages: ar-XA
SSML Gender: MALE
Name: ar-XA-Wavenet-D
Languages: ar-XA
SSML Gender: FEMALE
Name: bg-BG-Standard-A
Languages: bg-BG
SSML Gender: FEMALE
Name: bn-IN-Standard-A
Languages: bn-IN
SSML Gender: FEMALE
Name: bn-IN-Standard-B
Languages: bn-IN
SSML Gender: MALE
Name: bn-IN-Standard-C
Languages: bn-IN
SSML Gender: FEMALE
